In [1]:
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
df = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/component_training/df_train.csv")

In [23]:
df = df.iloc[100:105]

In [24]:
def get_summary(text):
    messages = [
    {"role": "system", "content": "You are a bug report summarizer. " \
     "Given a bug description that includes logs, stacktraces, your job is to summarize the bug in natural language for better understanding." \
     "During summarization avoid using special characters. Include as much crucial information as possible related to the bug."},
    {"role": "user", "content": "Sumamrize the bug report:\n" + text},
]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
    )
    
    return outputs[0]["generated_text"][len(prompt):]


In [25]:
summaries = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing issue..."):
    text = f"Bug Title: {row.issue_title}\nBug Description: {row.description}"
    sumamry = get_summary(text)
    sumamry = sumamry.replace("\n\n", "\n")
    summaries.append(sumamry)


df["summary"] = summaries
df.to_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/component_training/df_train_summarized.csv")

Processing issue...: 100%|██████████| 5/5 [00:16<00:00,  3.20s/it]


In [26]:
df_out = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/component_training/df_train_summarized.csv")

In [27]:
df_out

,Unnamed: 0.1,Unnamed: 0,issue_number,issue_title,description,issue_url,issue_state,creator,labels,owner,component,text,summary
0,100,1512,4208,Dockerfile for JDK11 installs gcc 7.4 by default,https://github.com/eclipse/openj9/blob/master/...,https://github.com/eclipse-openj9/openj9/issue...,closed,AdamBrousseau,comp:build,babsingh,comp:build,Title: Dockerfile for JDK11 installs gcc 7.4 b...,Here is a summarized version of the bug report...
1,101,7233,17910,OpenJDK java/lang/runtime/ReferencedKeyTest ja...,https://openj9-jenkins.osuosl.org/job/Test_ope...,https://github.com/eclipse-openj9/openj9/issue...,closed,pshipton,"comp:test, test failure",tajila,comp:test,Title: OpenJDK java/lang/runtime/ReferencedKey...,Here is a summary of the bug report in natural...
2,102,4640,12293,Add SCC option to log `verboseX` output to a file,Java -version output\r\n--------------------\r...,https://github.com/eclipse-openj9/openj9/issue...,open,DanHeidinga,"enhancement, comp:vm, userRaised",NaN,comp:vm,Title: Add SCC option to log `verboseX` output...,Here is a summarized version of the bug report...
3,103,2156,6284,Enable software CS testing on Power,I just noticed from [1] we have `^arch.ppc` sp...,https://github.com/eclipse-openj9/openj9/issue...,closed,fjeremic,"comp:test, arch:power, bits:64",gita-omr,comp:test,Title: Enable software CS testing on Power\nDe...,Here is a summarized version of the bug report...
4,104,4867,12784,Shared class cache name and number of connecte...,Requirement for the current JVM's resolved sha...,https://github.com/eclipse-openj9/openj9/issue...,closed,gvnmlln,"enhancement, comp:vm, userRaised",pshipton,comp:vm,Title: Shared class cache name and number of c...,Here is a summarized version of the bug report...


In [37]:
print(df.iloc[4].summary)

Here is a summarized version of the bug report in natural language:
The bug report requests that the name of the current JVM's resolved shared class cache and the number of connected users be made available through APIs. Currently, the resolved shared class cache name is not accessible through the openJ9 SharedClass APIs or the MemoryMXBean, but only the unresolved name can be obtained through the Java system API. The resolved cache name is important because CICS cannot resolve the symbols specified in the cache name. Additionally, the number of users connected to the shared class cache is already available from the openJ9 java core, but it is not accessible through the SharedClass APIs. This limitation prevents CICS from capturing this information and reporting it in its statistics report.


In [38]:
print(df.iloc[4].description)

Requirement for the current JVM's resolved shared class cache name and number of connected users to be added to the MemoryMXBean and SharedClass APIs (com.ibm.oti.shared.SharedClassStatistics and  com.ibm.oti.shared.SharedClassUtilities) so this information can be recorded in CICS JVM server statistics reports.

The resolved shared class cache name is not available today in the openJ9 SharedClass APIs or the MemoryMXBean. Only the unresolved shared class cache name is obtainable through the Java system API, via the -Xshareclasses:name=<> JVM option. The resolved cache name is valuable as CICS is unable to resolve the symbols that can be specified in the cache name. 

The number of users connected to the SCC is already available from the openJ9 java core, but it is not available from the SharedClass APIs. This limits CICS from capturing this information and reporting it in its statistics report. This information is valuable as it allows customers to confirm if the class cache is being s

In [39]:
print(df.iloc[4].issue_title)

Shared class cache name and number of connected users not available through APIs
